In [1]:
import pickle
import pandas as pd
import os
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch
import xml.etree.ElementTree as ET
import pandas as pd
from operator import itemgetter
from sklearn.preprocessing import OneHotEncoder

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
def form_getter(file_list, path):
    master_list = []

    for i in tqdm(file_list):
        root = ET.parse(i).getroot()
        track = root[0]
        date = root[1]
        for race in root.iter("Race"):
            print(race)
            ids = []
            boxes = []
            weights = []
            dog_name = []
            for dog in race.iter("Dog"):
                if dog[0].text not in "12345678":
                    print("longscratch")
                    continue
                else:
                    print(dog[0].text)


                if not dog.attrib["id"]:
                    continue
                if not dog[0].text:
                    continue
                if (dog[0].text in "RSBFNPTBD") or ("=" in dog[0].text):
                    # print("scratched check working")
                    continue
                else:
                    ids.append(dog.attrib["id"])
                    boxes.append(dog[0].text)
                    weights.append(dog[1].text)
                    dog_name.append(dog[2].text)


            zipall = list(
                zip(
                    ids,
                    boxes,
                    weights,
                    dog_name
                )
            )
            # print(zipall)
            for i in range(len(zipall)):
                zipall[i] = (
                    race.attrib["id"],
                    date.text,
                    track.text,
                    race[3].text,
                    race[4].text,
                ) + zipall[i]
                master_list.append(zipall[i])

    return master_list


In [3]:
def track_id_gen(dist, track):
    return track[0:4]+"-"+str(int(dist))

In [4]:
#pickle Load results
os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")
form_file = open( 'results-df-merged-prices.npy', 'rb')
form = pickle.load(form_file)
form

,race_id,date,track,dist_x,grade,place,dog_id,box,split_times,split_margins,...,PPMAX,PPMIN,IPMAX,IPMIN,MORNINGTRADEDVOL,PPTRADEDVOL,IPTRADEDVOL,<!DOCTYPE html>,dog,loc
432621,839820790,02 Nov 22,Traralgon,395.0,Restricted Win,5.0,558432636,5,3.86,0.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
137049,839820805,02 Nov 22,Geelong,400.0,Grade 7,4.0,507032541,3,8.87,0.21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
280198,839681516,02 Nov 22,Meadows (MEP),525.0,Maiden Heat,6.0,677865140,4,5.35,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
280199,839681516,02 Nov 22,Meadows (MEP),525.0,Maiden Heat,7.0,687504766,3,5.35,0.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
280200,839681516,02 Nov 22,Meadows (MEP),525.0,Maiden Heat,8.0,619722472,5,5.22,0.12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233494,1920060020,03 Jan 16,Healesville,300.0,S/E Veteran,3.0,-724496,2,4.10,0.19,...,1.0,1001.0,1.0,1001.0,5.00,0.0,0.0,NaN,Boom Boom Shiraz,AUS
233495,1920060020,03 Jan 16,Healesville,300.0,S/E Veteran,4.0,-693948,4,4.05,0.14,...,1.0,1001.0,1.0,1001.0,3.96,0.0,0.0,NaN,El Mania,AUS
233496,1920060020,03 Jan 16,Healesville,300.0,S/E Veteran,5.0,-727875,7,4.21,0.30,...,1.0,1001.0,1.0,1001.0,20.74,0.0,0.0,NaN,Oblivious,AUS
233497,1920060020,03 Jan 16,Healesville,300.0,S/E Veteran,6.0,-720926,3,4.08,0.17,...,1.0,1001.0,1.0,1001.0,19.52,0.0,0.0,NaN,Naughty And Nice,AUS


In [5]:
tracks = form.track.unique().tolist()

ohe = OneHotEncoder(dtype=int,sparse=False)
transformed = ohe.fit_transform(form[['track']])
#enc = OneHotEncoder( categories=tracks)


In [6]:
# file_list = ["ff-BEN07Oct22.xml"]
file_list = [r"forms\bf-BEN12Oct22.xml"]
fg = form_getter(file_list, path="")
fg

100%|██████████| 1/1 [00:00<00:00, 76.36it/s]

<Element 'Race' at 0x0000023C63E577C0>
1
2
3
4
5
6
7
8
longscratch
longscratch
<Element 'Race' at 0x0000023C63E79860>
1
2
3
4
5
6
7
8
longscratch
longscratch
<Element 'Race' at 0x0000023C63E8C860>
1
2
3
4
5
6
7
8
longscratch
longscratch
<Element 'Race' at 0x0000023C63EA48B0>
1
2
3
4
5
6
7
8
longscratch
longscratch
<Element 'Race' at 0x0000023C63EBB900>
1
2
3
4
5
6
7
8
longscratch
longscratch
<Element 'Race' at 0x0000023C63ED1950>
1
2
3
4
5
6
7
8
longscratch
longscratch
<Element 'Race' at 0x0000023C63EE8900>
1
2
3
4
5
6
7
8
longscratch
longscratch
<Element 'Race' at 0x0000023C63EFF810>
1
2
3
4
5
6
7
8
longscratch
longscratch
<Element 'Race' at 0x0000023C63F147C0>
1
2
3
4
5
6
7
8
longscratch
longscratch
<Element 'Race' at 0x0000023C63F2C770>
1
2
3
4
5
6
7
8
longscratch
longscratch
<Element 'Race' at 0x0000023C63F43770>
1
2
3
4
5
6
7
8
longscratch
longscratch
<Element 'Race' at 0x0000023C63F597C0>
1
2
3
4
5
6
7
8
longscratch
longscratch


[('832364915',
  '12 Oct 22',
  'Bendigo',
  '12 Oct 22 03:10AM',
  '425m',
  '537971299',
  '1',
  '58F36',
  'BARRY ALLAN'),
 ('832364915',
  '12 Oct 22',
  'Bendigo',
  '12 Oct 22 03:10AM',
  '425m',
  '626191396',
  '2',
  '74234',
  'WALDEN BALE'),
 ('832364915',
  '12 Oct 22',
  'Bendigo',
  '12 Oct 22 03:10AM',
  '425m',
  '675318980',
  '3',
  '644',
  'BROOKLYN STAR'),
 ('832364915',
  '12 Oct 22',
  'Bendigo',
  '12 Oct 22 03:10AM',
  '425m',
  '639219893',
  '4',
  '45256',
  'ISHMY BELLE'),
 ('832364915',
  '12 Oct 22',
  'Bendigo',
  '12 Oct 22 03:10AM',
  '425m',
  '639221391',
  '5',
  '64526',
  'SCOTT ME'),
 ('832364915',
  '12 Oct 22',
  'Bendigo',
  '12 Oct 22 03:10AM',
  '425m',
  '631806098',
  '6',
  '76676',
  'DALLAS JANE'),
 ('832364915',
  '12 Oct 22',
  'Bendigo',
  '12 Oct 22 03:10AM',
  '425m',
  '644453343',
  '7',
  '733',
  'DELINQUENT'),
 ('832364915',
  '12 Oct 22',
  'Bendigo',
  '12 Oct 22 03:10AM',
  '425m',
  '607054185',
  '8',
  'T4652',
  'ESPEC

In [7]:
pred_df = pd.DataFrame(fg, columns=["race_id","dateF", "track", "datetime", "dist", "dog_id", "box", "extra1", "dogname"])
pred_df["dist"] = pred_df["dist"].astype(str).str[:-1].astype(float)
pred_df = pred_df.sort_values('datetime')
pred_df

,race_id,dateF,track,datetime,dist,dog_id,box,extra1,dogname
0,832364915,12 Oct 22,Bendigo,12 Oct 22 03:10AM,425.0,537971299,1,58F36,BARRY ALLAN
1,832364915,12 Oct 22,Bendigo,12 Oct 22 03:10AM,425.0,626191396,2,74234,WALDEN BALE
2,832364915,12 Oct 22,Bendigo,12 Oct 22 03:10AM,425.0,675318980,3,644,BROOKLYN STAR
3,832364915,12 Oct 22,Bendigo,12 Oct 22 03:10AM,425.0,639219893,4,45256,ISHMY BELLE
4,832364915,12 Oct 22,Bendigo,12 Oct 22 03:10AM,425.0,639221391,5,64526,SCOTT ME
...,...,...,...,...,...,...,...,...,...
90,832364911,12 Oct 22,Bendigo,12 Oct 22 06:57AM,425.0,444701628,3,7F666,BIT TWISTED
91,832364911,12 Oct 22,Bendigo,12 Oct 22 06:57AM,425.0,403642187,4,57785,VORTEX
92,832364911,12 Oct 22,Bendigo,12 Oct 22 06:57AM,425.0,307563901,5,66647,MCIVOR BLUEY
93,832364911,12 Oct 22,Bendigo,12 Oct 22 06:57AM,425.0,538510873,6,62666,MISS BAZAAR


In [8]:
pred_df.columns

ohe = OneHotEncoder(dtype=int,sparse=False)
transformed = ohe.fit(form[['track']])

transformed.transform(pred_df[['track']])

pred_df["tracksOnehot"] =  transformed.transform(pred_df[['track']]).tolist()
pred_df["tracksOnehot"]

0     [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1     [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
2     [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
3     [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
4     [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
                          ...                      
90    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
91    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
92    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
93    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
95    [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Name: tracksOnehot, Length: 96, dtype: object

In [9]:
race_forms = form.groupby(["race_id"])
dog_forms = form.groupby(["dog_id"])
print(race_forms)
race_input_layer = []
race_classes = []
full_details = []

race_pred_forms = pred_df.groupby(["race_id"], sort=False)

# for i, j in tqdm(race_forms):
#     print( f"{i=},{j=}")
#     break


for i, j in tqdm(race_pred_forms):
    
        race_date = pd.to_datetime(j["dateF"].iloc[0])
        
        # print(f"{race_date=}")
        dogs = j.dog_id
        #print(f"{j.dogname.to_list()=}")
        # prev_form = form[(form["date"]< race_date)]
        race_id = i
        race_time = j.datetime.iloc[0]
        # print(prev_form.shape)
        race_dist = j.dist.iloc[1]
        trackOHE = j.tracksOnehot.iloc[1]
        dog_info = []
        dog_names = []
        print(race_time)

        i = 0
        for dog in dogs:
            
            dog_box = j.box.iloc[i]
            #print(f"{dog_box=}")
            dog_name = j.dogname.iloc[i]
            if not dog_name:
                dog_name = "scratch"
            dog_names.append(dog_name)
            #print(f"{dog_name=}")
            #print(f"{dog=}")
            try:
                prev_form = dog_forms.get_group(dog)
                #print("got here")            
                prev_form = prev_form[(prev_form["dateF"] < race_date)]
                prev_races = prev_form.shape[0]
                dog_speed_avg = prev_form.speed.mean()
                dog_speed_sd = prev_form.speed.std()
                dog_speed_max = prev_form.speed.max()
                dog_split_avg = prev_form.split_speed.mean()
                dog_split_sd = prev_form.split_speed.std()
                dog_split_max = prev_form.split_speed.max()
                dog_split_mg_avg = prev_form.split_margins.mean()
                dog_margin_mean = prev_form.margin.mean()
                dog_margin_sd = prev_form.margin.std()
                dog_race_count = prev_form.shape[0]
                dog_wins = prev_form[prev_form["place"] == 1].shape[0]
                dog_places = prev_form[prev_form["place"] <= 3].shape[0]
                dog_weight = prev_form.weigth.iloc[0]
                if prev_races:
                    dog_margin_last = prev_form.margin.iloc[0]
                    dog_speed_last = prev_form.speed.iloc[0]
                else:
                    dog_margin_last = -1
                    dog_speed_last = -1
                dog_stats = (
                    dog,
                    [
                        dog_box,
                        dog_speed_avg,
                        dog_speed_sd,
                        dog_speed_max,
                        dog_split_avg,
                        dog_split_sd,
                        dog_split_max,
                        dog_split_mg_avg,
                        dog_margin_mean,
                        dog_margin_sd,
                        dog_race_count,
                        dog_wins,
                        dog_places,
                        dog_weight,
                        dog_speed_last,
                        dog_margin_last
                    ],
                )
                
                #p#rint()
            except Exception as e:
                dog_stats = (
                    dog,
                    [dog_box]+[-1]*15
                )
                print(e)
                #dog_stats = (dog,[])
            dog_info.append(dog_stats)
            # print(f"{dog_stats=}")
            i = i + 1

        dog_info.sort(key=itemgetter(0))
        input_layer = (
            trackOHE
            + [race_dist]
            + [item for sublist in [x[1] for x in dog_info] for item in sublist]
        )
        if len(dogs) > 8:
            print(f"{race_id=}")
            print(f"{race_date=}")
            print(f"{j.track.iloc[1]=}")
        race_input_layer.append(input_layer)
        full_details.append(
            (race_id, input_layer, dog_names))



  8%|▊         | 1/12 [00:00<00:01,  6.79it/s]

12 Oct 22 03:10AM
12 Oct 22 03:30AM
12 Oct 22 03:54AM
12 Oct 22 04:17AM
12 Oct 22 04:34AM


100%|██████████| 12/12 [00:00<00:00, 32.31it/s]

12 Oct 22 04:52AM
12 Oct 22 05:17AM
12 Oct 22 05:38AM
12 Oct 22 05:59AM
12 Oct 22 06:19AM
12 Oct 22 06:39AM
12 Oct 22 06:57AM


In [10]:
with open("prediction_inputs.npy", "wb") as fp:   #Pickling
    
    pickle.dump(full_details, fp)

In [ ]:
dfpred = pd.DataFrame(dog_)

In [11]:
test_df = pd.DataFrame(race_input_layer)